In [ ]:
#%matplotlib notebook
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(9,4.5))
rc('font',size=12)
rc('text',usetex=False)

from qubicpack.qubicfp import qubicfp
import qubic.fibtools as ft
import qubic.plotters as p
import qubic.lin_lib as ll
import qubic.demodulation_lib as dl
import satorchipy as stpy
from pysimulators import FitsArray

import numpy as np
from matplotlib.pyplot import *
import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
import glob
import string
import scipy.signal as scsig
from scipy import interpolate
import datetime as dt
import pickle
from importlib import reload

This is the scrpt that analyses the data files corresponding to various HWP datasets. They are each with a different configuration of the calibration source. The two last ones have a slight change in the pointing w.r.t. the first three so this could induce some effect.
This scripts produces data that are then analyzed by the script called: Analyse-HWP-Results.Rmd

This is still work in progress so some things are still to be done:
- the anaysis is only done so far with the simulated calibration source, not the actual CalSrc data, this can induce significant effects so should be explored.

# Setting the directory containing data

Here are the vailable data:

### 2019-12-23:
- 0: 2019-12-23_18.16.06__HWP_Scanning_Vtes_2TimePerPos_10: 
    - TES data is bad, probably bad TES COnfiguration
- 1: 2019-12-23_18.26.04__HWP_Scanning_Vtes_2TimePerPos_60: 
    - Same
- 2: 2019-12-23_18.52.34__HWP_Scanning_Vtes_2TimePerPos_60: 
    - Same + Interrupted shortly after start
- 3: 2019-12-23_19.00.20__HWP_Scanning_Vtes_2TimePerPos_60: 
    - Same + Interrupted shortly after start
- 4: 2019-12-23_19.04.56__HWP_Scanning_Vtes_2TimePerPos_60: 
    - Nice data although source data does not have the same shape as TES data, probably the source measurement was not configured correctly. The data can however be exploited using Simulated Cal Src.

### 2019-12-24:
- 0: 2019-12-24_09.46.19__HWP_Scanning_Vtes_2TimePerPos_180:
    - Nice data with 180 sec/pos and 3 cycles.
    - SrcData not there...
    - Can be used with Simulated Cal Src

### 2019-12-26:
- 0: 2019-12-26_16.04.14__HWP_Scanning_Vtes_2TimePerPos_60:
    - Only one cycle but good quality data, The source is ON and seems weell configured
- 1: 2019-12-26_18.42.46__HWP_Scanning_Vtes_2TimePerPos_900:
    - Long overnight acquisition - to be looked at closely

### 2019-12-27:
- 0: 2019-12-27_11.45.33__HWP_Scanning_Vtes_2TimePerPos_60:
    - too short (interrupted)
- 1: 2019-12-27_11.50.24__HWP_Scanning_Vtes_2_TimePerPos_60_Cylce_0_over_5:
    - too short (interrupted)
- 2: 2019-12-27_11.51.37__HWP_Scanning_Vtes_2_TimePerPos_60_Cycle_0_over_5:
    - Excellent data
- 3: 2019-12-27_11.58.51__HWP_Scanning_Vtes_2_TimePerPos_60_Cycle_1_over_5:
    - HWP axis broken at the end of the previous scan... Not usable...
- 4: 2019-12-27_12.06.06__HWP_Scanning_Vtes_2_TimePerPos_60_Cycle_2_over_5:
    - Not usable...
- 5: 2019-12-27_12.13.23__HWP_Scanning_Vtes_2_TimePerPos_60_Cycle_3_over_5:
    - Not usable...
- 6: 2019-12-27_12.20.42__HWP_Scanning_Vtes_2_TimePerPos_60_Cycle_4_over_5:
    - Not usable...
- 7: 2019-12-27_12.57.14__HWP_Scanning_Vtes_2_TimePerPos_60_Cycle_0_over_1:
    - Not usable...

## So finally the good data are:
### 2019-12-23: 4
- Modulator: Amplitude = 2V. Offset = 1.5 V
- Nice data although source data does not have the same shape as TES data, probably the source measurement was not configured correctly. The data can however be exploited using Simulated Cal Src.

### 2019-12-24: 0
- Modulator: Amplitude = 500 mV ; Offest = 250 mV
- Nice data with 180 sec/pos and 3 cycles.
- SrcData not there...
- Can be used with Simulated Cal Src

### 2019-12-26: 0
- Modulator: Amplitude: 500mV, Offset 2. V
- Only one cycle but good quality data, The source is ON and seems weell configured

### 2019-12-26: 1
- Modulator: Amplitude: 500mV, Offset 2.5 V
- Long overnight acquisition - to be looked at closely

### 2019-12-27: 2
- Modulator: Amplitude = 500mV ; Offest = 2.5 V
- Excellent data

In [ ]:
day = '2019-12-24'
keyword = '*HWP_Scanning*'
#data_dir = '/qubic/Data/Calib-TD/'+day+'/'
data_dir = '/Users/hamilton/Qubic/Calib-TD/'+day+'/'
dirs = np.sort(glob.glob(data_dir+keyword))

print (dirs)


### Quick Look at the data
Here we select one of the directories and have alook at the data. We will check the signal and define the modulation frequency, filtering to be applied and so on...

In [ ]:
ifile = 0
thedir = dirs[ifile]
print(thedir)
a = qubicfp()
a.read_qubicstudio_dataset(thedir)
a.timeaxis(asic=1,datatype='sci',axistype='index')
data=a.azel_etc(TES=None)
t0=data['t_data 2'][0]

hwp = a.hk['INTERN_HK']['HWP-Position']
hwp_timestamps = a.timeaxis(datatype='hk',axistype='computertime')

print(a.hk.keys())
print(a.hk['INTERN_HK'].keys())
print(a.hk['CALSOURCE-CONF'].keys())

data.keys()

Quick look at the data picking one TES in one asic:
- Do we see the HWP going through various positions ?
- Do we see the calibration source ? If so, how well is it synchronized with the data signal (this is why we dispaly a zoom of both)
    - If the calibration source is present but badly synchrinized, we will need to shift it in time to have it in phase with the data
    - If it is not present, we will need to use the "Cal Src Simulator" and fit the time shift to have it in phase with the data

In [ ]:
rc('figure',figsize=(12,12))
TESNum = 95
asic = str(1)

### Interpolating HWP data to the timestamps of the data
hwp_data = np.interp((data['t_data '+asic]-t0), (hwp_timestamps-hwp_timestamps[0]), hwp)
### Interpolating source data to the timestamps of the data
data_src_interp = np.interp(data['t_data '+asic], data['t_src'], data['data_src'])

subplot(2,1,1)
plot((data['t_data '+asic]-t0), dl.renorm(data['data '+asic][TESNum-1,:]), 
     label='Data TES {} ASIC {}'.format(TESNum,asic))
plot((data['t_data '+asic]-t0), hwp_data, label='HWP Position')
plot((data['t_src']-t0), dl.renorm(data['data_src'])-5, label='source')
xlabel('Seconds')
legend()
title(thedir)
ylim(-10,10)

subplot(2,1,2)
oktime = np.abs((data['t_data '+asic]-t0)-100) < 10
plot((data['t_data '+asic]-t0)[oktime], dl.renorm(data['data '+asic][TESNum-1,oktime]), 
     label='Data TES {} ASIC {}'.format(TESNum,asic))
plot((data['t_data '+asic]-t0)[oktime], hwp_data[oktime], label='HWP Position')
plot((data['t_data '+asic]-t0)[oktime], dl.renorm(data_src_interp[oktime]), label='SRC (interp on data)')
#plot((data['t_src']-t0), dl.renorm(data['data_src']), label='source')
xlabel('Seconds')
legend()
title(thedir)
ylim(-10,10)




We have a look at the power spectrum of the source signal, and set the filtering and source modulaiton frequency

In [ ]:
rc('figure',figsize=(13,8))
#### Filtering
reload(ft)
tt = data['t_data '+asic]
t0 = tt[0]
dd = data['data '+asic][TESNum-1,:]

thefreqmod = 1.

period = 1./ thefreqmod
lowcut = 0.8
highcut = 70

xmin = 0.01
xmax = 90.
ymin = 1e1
ymax = 1e17

############ Power spectrum
spectrum_f, freq_f = ft.power_spectrum(tt, dd, rebin=True)
plot(freq_f, f.gaussian_filter1d(spectrum_f,1),label='Raw Data')
yscale('log')
xscale('log')
xlabel('Frequency [Hz]')
ylabel('Power Spectrum')
xlim(xmin, xmax)
ylim(ymin, ymax)
title('TES {} ASIC {}'.format(TESNum,asic))

for i in range(10):
    plot([1./period*i,1./period*i],[ymin,ymax],'k--',alpha=0.3)

plot([lowcut, lowcut],[ymin,ymax],'k')
plot([highcut, highcut],[ymin,ymax],'k')
legend()

########## New Power spectrum
nharm = 10
notch = np.array([[1.724, 0.005, nharm]])
newdata = ft.filter_data(tt, dd, lowcut, highcut, notch=notch, rebin=True, verbose=True, order=5)
spectrum_f2, freq_f2 = ft.power_spectrum(tt, newdata, rebin=True)
plot(freq_f2, f.gaussian_filter1d(spectrum_f2,1),label='Filtered data')
for i in range(nharm):
    plot([notch[0,0]*(i+1),notch[0,0]*(i+1)], [ymin,ymax],'m:')
legend()

tight_layout()

From the HWP position measured by QUBIC Studio we defin the various data chunks each corresponding to a given HWP position. We also cut a fraction of the data to let the HWP reach a stable position. We remove situations where the HWP data is 255 which means it has not been read by QUBIC Studio.

In [ ]:
hwp_chunks = []
hwp_pos_chunk = []
istart = 0
tocut = 2000
hwp_pos = hwp_data[istart]
for i in range(len(hwp_data)):
    if (hwp_data[i] == int(hwp_data[i])):
        if (hwp_data[i] != hwp_data[istart]):
            if hwp_data[istart] != 255:
                hwp_chunks.append([istart+tocut, i-tocut])
                hwp_pos_chunk.append(int(hwp_data[istart]))
            istart = i
            #print(hwp_pos_chunk)

        
print(np.unique(np.array(hwp_pos_chunk)))

rc('figure',figsize=(13,8))
cols = ['r','b','g','k','m','y','orange']
angles = np.arange(7)*15
thvals = np.zeros(len(hwp_chunks))
for i in range(len(hwp_chunks)):
    chunk = hwp_chunks[i]
    plot((data['t_data '+asic][chunk[0]:chunk[1]]-t0)/3600, newdata[chunk[0]:chunk[1]], 
        color=cols[hwp_pos_chunk[i]-1])
    thvals[i] = angles[hwp_pos_chunk[i]-1]

print(thvals)    

xlim(0,np.max((data['t_data '+asic]-t0)/3600))
mm,ss = ft.meancut(newdata,3)
ylim(mm-2*ss,mm+3*ss)
for i in arange(1,8):
    plot(-1,-1, 'o',color=cols[i-1], label='HWP Pos = {} ; Angle = {} deg.'.format(i, angles[i-1]))
legend()
title('TES {} ASIC {}'.format(TESNum,asic))

If the CalSrc data was ot good, we use the CalSrc simulator in the cell below and superinpose it with the data

In [ ]:
for k in a.hk['CALSOURCE-CONF'].keys():
    print(k+': {}'.format(a.hk['CALSOURCE-CONF'][k]))

In [ ]:
print(a.hk['CALSOURCE-CONF']['Mod_ampl']/1000,a.hk['CALSOURCE-CONF']['Mod_offs']/1000)


# #### Source characteristics
rf_freq = a.hk['CALSOURCE-CONF']['Cal_freq']     # GHz
amplitude = a.hk['CALSOURCE-CONF']['Mod_ampl']/1000 *0+0.5    # Volts
offset = a.hk['CALSOURCE-CONF']['Mod_offs']/1000*0+1        # volts
phase = 0        # radians
frequency = a.hk['CALSOURCE-CONF']['Mod_freq']#/1000    # Hz [Modulation Frequency] 
print(amplitude, offset, frequency, phase, rf_freq)
signal_src = -ll.sim_generator_power(tt, amplitude, offset, frequency, phase, rf_freq = rf_freq)
print(len(tt), len(signal_src))

rc('figure',figsize=(13,8))
ichunk = 0
chunk = hwp_chunks[ichunk]
newtt = (data['t_data '+asic][chunk[0]:chunk[1]]-t0)
newdd = newdata[chunk[0]:chunk[1]]
ddsrc = -data_src_interp[chunk[0]:chunk[1]]
sig_src = signal_src[chunk[0]:chunk[1]]

subplot(2,1,1)
plot(newtt-newtt[0],dl.renorm(newdd),label='Data TES {} ASIC {}'.format(TESNum, asic))
plot(newtt-newtt[0],-dl.renorm(ddsrc),label='CalSrc [time shifted]')
plot(newtt-newtt[0],-dl.renorm(sig_src),label='Simulated CalSrc [Phase manually set]')
title('Pos={}'.format(hwp_pos_chunk[ichunk]))
xlim(0,10)
legend()

subplot(2,1,2)
spectrum_f2, freq_f2 = ft.power_spectrum(newtt, newdd, rebin=True)
plot(freq_f2, f.gaussian_filter1d(spectrum_f2,1),label='Filtered data')
for i in range(10):
    plot([1./period*i,1./period*i],[ymin,ymax],'k--',alpha=0.3)
# for i in range(nharm):
#     plot([notch[0,0]*(i+1),notch[0,0]*(i+1)], [ymin,ymax],'m:')
yscale('log')
xscale('log')
xlabel('Frequency [Hz]')
ylabel('Power Spectrum')
xlim(xmin, xmax)
ylim(ymin, ymax)
legend()

### Proper analysis
We prepare the data for the source simulator, or for the source itself. Various demodulation techniques are available.

In [ ]:
reload(dl) 

##### Filtering to be used
lowcut = 0.8
highcut = 70
nharm = 10
notch = np.array([[1.724, 0.005, nharm]])
fourier_cuts = [lowcut, highcut, notch]


vals_demod = np.zeros((len(hwp_chunks),256))
err_demod = np.zeros((len(hwp_chunks),256))
vals_demod_quad = np.zeros((len(hwp_chunks),256))
err_demod_quad = np.zeros((len(hwp_chunks),256))

for i in range(len(hwp_chunks)):
    print('Doing Chunk {} over {}'.format(i, len(hwp_chunks)))
    chunk = hwp_chunks[i]
    if (chunk[0] < chunk[1]):
        for asic in [1,2]:
            ### TES data --------------------------------------------------------------------------
            tod = data['data {}'.format(asic)][:,chunk[0]:chunk[1]]
            time = data['t_data {}'.format(asic)][chunk[0]:chunk[1]]-np.min(data['t_data {}'.format(asic)][chunk[0]:chunk[1]])
            deltat = (time[-1]-time[1])   # Chunk duration
            print('  - Asic {} - Duration {} seconds'.format(asic, deltat))
            #--------------------------------------------------------------------------------------

            ### Only analyse chinks whose duration is larger than a given number of seconds
            if  (deltat > 5.):
                ### Source data -----------------------------------------------------------------------
                #ddsrc = -data_src_interp[chunk[0]:chunk[1]]
                #--------------------------------------------------------------------------------------

            
                ### Source Simulated Data -------------------------------------------------------------
                # Find best phase for Simulated Source: we use a reference TES and check which phases
                # leads to the best agreement with TES data (renormalized to mean 0 and RMS 1)
                print('    Fitting best phase for Calibration Source')
                TES_reference = TESNum
                # Filter TES
                TES_ref_data = dl.renorm(ft.filter_data(time, tod[TES_reference-1,:], lowcut, highcut, notch=notch, 
                                                        rebin=True, verbose=False, order=5))
                # Loop over a number of phases, calculate the src sim with a phase and compute chi2 with TES data
                nphases = 100
                phases = np.linspace(0, 2*np.pi, nphases)
                ch2 = np.zeros(nphases)
                for k in range(nphases):
                    phase = phases[k]
                    # Minus sign as source responds negatively
                    signal_src_sim = -ll.sim_generator_power(time, amplitude, offset, frequency, phase, rf_freq = rf_freq)
                    # Filter the source the same way as TES data
                    signal_src_sim = ft.filter_data(time, signal_src_sim, lowcut, highcut, notch=notch, 
                                                            rebin=True, verbose=False, order=5)
                    # Renormalize it the same way as the data
                    signal_src_sim = dl.renorm(signal_src_sim)
                    # Calculate Ch2
                    ch2[k] = np.sum((TES_ref_data-signal_src_sim)**2)
                # Take the smallest chi2 as the source simulation phase
                best_phase = phases[np.argmin(ch2)]
                print('    Found that best phase for this chunk is: {}'.format(best_phase))
                ### Minus sign as source responds negatively
                signal_src_sim = -dl.renorm(ll.sim_generator_power(time, amplitude, offset, frequency, best_phase, rf_freq = rf_freq))
                src_data = [time, signal_src_sim]
                #--------------------------------------------------------------------------------------


                #### demodulation method
                print('     - Demodulation Method')
                newt_demod, amp_demod, errors_demod = dl.demodulate_methods([time, tod],
                                                                            1./period, 
                                                                            src_data_in=src_data,
                                                                            method='demod',
                                                                           fourier_cuts=fourier_cuts)
                vals_demod[i,128*(asic-1):128*(asic-1)+128] = -np.median(amp_demod, axis=1)
                err_demod[i,128*(asic-1):128*(asic-1)+128] = np.std(amp_demod, axis=1)/np.sqrt(len(amp_demod))

                #### Quadrature demodulation method
#                 print('     - Quadrature Demodulation Method')
#                 newt_demod_quad, amp_demod_quad, errors_demod_quad = dl.demodulate_methods([time, tod],
#                                                                             1./period, 
#                                                                             src_data_in=src_data,
#                                                                             method='demod_quad', remove_noise=True,
#                                                                            fourier_cuts=fourier_cuts)
#                 vals_demod_quad[i,128*(asic-1):128*(asic-1)+128] = -np.median(amp_demod_quad, axis=1)
#                 err_demod_quad[i,128*(asic-1):128*(asic-1)+128] = np.std(amp_demod_quad, axis=1)/np.sqrt(len(amp_demod_quad))





### Regroup identical HWP position together with weighted average

In [ ]:
uniqth = np.unique(np.sort(thvals))
vals_demod_uniq = np.zeros((256, len(uniqth)))
err_demod_uniq = np.zeros((256, len(uniqth)))
for i in range(256):
    ok = -vals_demod[:,i] !=0
    tt = thvals[ok]
    dd = -vals_demod[ok,i]
    ee = err_demod[ok,i]
    for k in range(len(uniqth)):
        thisone = tt == uniqth[k]
        vals_demod_uniq[i,k], err_demod_uniq[i,k] = ft.weighted_mean(dd[thisone], ee[thisone], dispersion=False)




### Show result for one TES

In [ ]:
reload(dl)
theTES = 95
asic = 1
iii = (asic-1)*128 + (theTES-1)

myguess = np.array([np.max(abs(vals_demod_uniq[iii,:])), 0., 0., 5.])
resfit = dl.hwp_fitpol(uniqth, vals_demod_uniq[iii,:], err_demod_uniq[iii,:], 
                       doplot=True, str_title='Demodulation TES{} ASIC{}'.format(iii+1,asic), 
                       saturation=True, myguess=myguess, force_chi2_ndf=True)
tight_layout()
print(resfit[1])

### Loop on all TES

In [ ]:
day+'_Dataset_{}'.format(ifile)

In [ ]:
thvals.shape
vals_demod.shape
err_demod.shape

In [ ]:
rc('figure',figsize=(6,4.5))
reload(dl)
dir_pictures = '/Users/hamilton/Qubic/Calib-TD/HWP-Fitting/'+day+'_Dataset_{}'.format(ifile)+'/Pictures/'
dir_data = '/Users/hamilton/Qubic/Calib-TD/HWP-Fitting/'+day+'_Dataset_{}'.format(ifile)+'/Data/'
os.makedirs(dir_pictures, exist_ok=True)
os.makedirs(dir_data, exist_ok=True)

results = np.zeros((4, 256))
errors = np.zeros((4, 256))
chi2 = np.zeros(256)
ndf = np.zeros(256)
for iii in range(256):
    clf()
    myguess = np.array([np.max(abs(vals_demod_uniq[iii,:])), 0., 0., 5.])
    resfit = dl.hwp_fitpol(uniqth, vals_demod_uniq[iii,:], err_demod_uniq[iii,:], 
                           doplot=True, str_title='Demodulation TES{} ASIC{}'.format(iii+1,asic), 
                           saturation=True, myguess=myguess, force_chi2_ndf=True)
    results[:,iii] = resfit[1]
    errors[:,iii] = resfit[2]
    chi2[iii] = resfit[4]
    ndf[iii] = resfit[5]
    tight_layout()
    np.savetxt(dir_data+'/hwp_measurement_UniqueTh_TES_{}.txt'.format(iii+1),
               np.array([uniqth, vals_demod_uniq[iii,:], err_demod_uniq[iii,:]]).T)
    np.savetxt(dir_data+'/hwp_measurement_AllTh_TES_{}.txt'.format(iii+1),
               np.array([thvals, vals_demod[:,iii], err_demod[:,iii]]).T)
    np.savetxt(dir_data+'/hwp_measurement_Fit_UniqueTh_TES_{}.txt'.format(iii+1),
               np.array([resfit[1], resfit[2]]).T)
    savefig(dir_pictures+'/hwp_measurement_TES_{}.png'.format(iii+1))
    

In [ ]:
rc('figure',figsize=(14,8))
reload(dl)
allTES = [93, 94, 95, 96]
i = 0
for theTES in allTES:
    subplot(2,2,i+1)
    asic = 1
    iii = (asic-1)*128 + (theTES-1)
    myguess = np.array([np.max(abs(vals_demod_uniq[iii,:])), 0., 0., 5.])
    resfit = dl.hwp_fitpol(uniqth, vals_demod_uniq[iii,:], err_demod_uniq[iii,:], 
                       doplot=True, str_title='Demodulation TES{} ASIC{}'.format(iii+1,asic), 
                       saturation=True, myguess=myguess, force_chi2_ndf=True)
    tight_layout()
    i=i+1

    
    
    

#### Now show the whole Focal Plane

In [ ]:
rc('figure',figsize=(14,8))
subplot(2,3,1)
plot(results[0,:])
xlabel('Amplitude')
subplot(2,3,2)
plot(results[1,:])
xlabel('XPol')
subplot(2,3,3)
plot(results[2,:])
xlabel('Phase')
subplot(2,3,4)
plot(results[3,:])
xlabel('Saturation')
subplot(2,3,5)
plot(np.log10(chi2))
xlabel('Log10(Chi2)')


In [ ]:
pull = (results[2,:]-np.median(results[2,:]))/errors[2,:]
ok = (np.product(errors, axis=0) != 0) & (np.abs(pull) < 4) & (np.abs(results[2,:]-np.median(results[2,:])) < 50) & (np.abs(np.log10(chi2)-1) < 1 )

rc('figure',figsize=(14,8))
dens = False
subplot(2,3,1)
hist(np.log10(results[0,:]), bins=30, density=dens, alpha=0.5, range=[1,4])
hist(np.log10(results[0,ok]), bins=30, density=dens, alpha=0.5, range=[1,4])
xlabel('Amplitude Log10')
subplot(2,3,2)
hist(np.log10(results[1,:]*100), bins=30, density=dens, alpha=0.5, range=[-6,3])
hist(np.log10(results[1,ok]*100), bins=30, density=dens, alpha=0.5, range=[-6,3])
xlabel('Log(XPol [%]) ')
subplot(2,3,3)
hist(results[2,:],bins=30, density=dens, alpha=0.5, range=[-30,30])
hist(results[2,ok],bins=30, density=dens, alpha=0.5, range=[-30,30])
xlabel('Phase')
subplot(2,3,4)
hist(np.log10(results[3,:]), bins=30, density=dens, alpha=0.5, range=[-4,2])
hist(np.log10(results[3,ok]), bins=30, density=dens, alpha=0.5, range=[-4,2])
xlabel('Saturation Log10')
subplot(2,3,5)
hist(np.log10(chi2), bins=30, density=dens, alpha=0.5, range=[-2, 4])
hist(np.log10(chi2[ok]), bins=30, density=dens, alpha=0.5, range=[-2, 4])
xlabel('Log10(Chi2)')


In [ ]:
### Manual selection of OK TES for file 24/12/2019
TESNUMOK = [1,2,4,6, 8, 10, 11, 12, 13, 14, 17, 21, 24, 26, 29, 30, 31, 33, 35, 38, 42, 44, 46, 48, 49, 53, 56, 60, 61, 62, 63, 64, 65, 66, 67, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, ]

In [ ]:
myamplitude = results[0,:].copy()
myamplitude[~ok] = np.nan

mycrosspol = np.log10(100*results[1,:]).copy()
mycrosspol[~ok] = np.nan

myphase = results[2,:].copy()
myphase[~ok] = np.nan

mysat = results[3,:].copy()
mysat[~ok] = np.nan

subplot(2,2,1)
img = ft.image_asics(all1=myamplitude)
imshow(img)
colorbar()
title('Amplitude')
subplot(2,2,2)
img = ft.image_asics(all1=mycrosspol)
imshow(img)
colorbar()
title('Log(Cross-Pol [%])')
subplot(2,2,3)
img = ft.image_asics(all1=myphase)
imshow(img, vmin=-180, vmax=180)
colorbar()
title('Phase')
subplot(2,2,4)
img = ft.image_asics(all1=mysat)
imshow(img)
colorbar()
title('Saturation')
